In [1]:
import json
import random
import warnings
from pathlib import Path

import load_data
import numpy as np
import stylized_score
import train_fingan

## Set seeds for reconstruction of the results

In [2]:
# Set seeds.
SEED = 12345
np.random.seed(SEED)
random.seed(SEED)

#### Load Data

Filter out selected times and stocks

In [3]:
min_len = 9216
B = 500
S = 8
L = 4096
price_df = load_data.load_prices("sp500")
log_ret, symbols = load_data.get_log_returns(price_df, min_len, True)
stf = stylized_score.boostrap_stylized_facts(log_ret, B, S, L)

Timespan 1987-12-17 -- 2024-07-17
Number of symbols 234


In [4]:
# get data
run_dir = Path("../data/fingan_runs")
experiments = []
for i, path in enumerate(run_dir.iterdir()):
    if (path / "final/model.pt").exists():
        print(f"Working on {str(path)}, nr {i}")

        try:
            fingan_model = train_fingan.load_fingan(path / "final/model.pt")
            with (path / "final/model_info.json").open() as file:
                info = json.load(file)

            def sampler(S):
                with warnings.catch_warnings(action="ignore"):
                    return train_fingan.sample_fingan(fingan_model, S)

            m_stf = stylized_score.stylied_facts_from_model(sampler, B, S)
            tot, ind, _ = stylized_score.stylized_score(stf, m_stf)

            experiments.append(
                {
                    "score": tot,
                    "ind_scores": ind,
                    "path": str(path),
                    "name": path.stem,
                    "dist": info["dist"],
                    "mark": "\\xmark"
                    if len(info["stylized_losses"]) == 0
                    else "\\cmark",
                }
            )
        except:  # noqa E722
            print(f"Failed {str(path)}")

        experiments.sort(key=lambda x: x["score"])
        with (run_dir / "fingan_experiments.json").open("w") as file:
            file.write(json.dumps(experiments, ensure_ascii=True, indent=4))

Working on ../data/fingan_runs/FinGanTakahashi_2024_08_20-15_39_37, nr 0
Working on ../data/fingan_runs/FinGanTakahashi_2024_08_20-15_40_26, nr 1
Working on ../data/fingan_runs/FinGanTakahashi_2024_08_20-15_40_20, nr 2
Working on ../data/fingan_runs/FinGanTakahashi_2024_08_20-15_32_37, nr 3
Working on ../data/fingan_runs/FinGanTakahashi_2024_08_20-15_40_32, nr 4
Working on ../data/fingan_runs/FinGanTakahashi_2024_08_20-15_31_34, nr 6


In [7]:
print_experiemnts = []
for exp in experiments:
    print_experiemnts.append(
        (exp["score"], exp["ind_scores"], exp["dist"], exp["mark"])
    )

print_experiemnts = sorted(print_experiemnts, key=lambda x: x[0])

# Print Tables for thesis

In [8]:
table_start = r"""\begin{table}
  \centering
  \begin{threeparttable}
    \caption{FinGAN model scores}
    \label{tab:finGAN}
        \begin{tabular}{l|c|l|l|l|l|l|l|l}
            \toprule
            distribution & stylized losses & $\mathcal{S}$ & $\mathcal{S}^{lu}$ & $\mathcal{S}^{ht}$
                         & $\mathcal{S}^{vc}$ & $\mathcal{S}^{le}$ & $\mathcal{S}^{cf}$ & $\mathcal{S}^{gl}$\\"""
table_end = """            \\bottomrule
        \\end{tabular}
        \\begin{tablenotes}[flushleft]
            \\footnotesize
            \\item FinGAN model scores. The model distribution was obtained by resampling 500 times
            eight sequences of lenght 4096 and computing the averaged estimators for the stylized facts
            over the eight samples. The score is obtained by a comparison of the bootstrapped real distribution
            and the model distribution obtained by resampling.
        \\end{tablenotes}
  \\end{threeparttable}
\\end{table}"""

print(table_start)
for mu, scs, model, mark in print_experiemnts:
    ind_scores_str = " & ".join([f"${d:.2f}$" for d in scs])
    print(r"            \hline")
    print(f"            {model} & {mark} & {mu:.2f} & {ind_scores_str}\\\\")

print(table_end)

\begin{table}
  \centering
  \begin{threeparttable}
    \caption{FinGAN model scores}
    \label{tab:finGAN}
        \begin{tabular}{l|c|l|l|l|l|l|l|l}
            \toprule
            distribution & stylized losses & $\mathcal{S}$ & $\mathcal{S}^{lu}$ & $\mathcal{S}^{ht}$
                         & $\mathcal{S}^{vc}$ & $\mathcal{S}^{le}$ & $\mathcal{S}^{cf}$ & $\mathcal{S}^{gl}$\\
            \hline
            laplace & \cmark & 0.57 & $1.26$ & $0.51$ & $0.59$ & $0.58$ & $0.22$ & $0.27$\\
            \hline
            studentt & \cmark & 0.67 & $1.37$ & $0.36$ & $0.69$ & $0.61$ & $0.39$ & $0.59$\\
            \hline
            normal & \cmark & 0.71 & $1.30$ & $0.32$ & $1.08$ & $0.72$ & $0.31$ & $0.51$\\
            \hline
            studentt & \xmark & 1.97 & $2.62$ & $0.38$ & $2.83$ & $2.10$ & $2.63$ & $1.29$\\
            \hline
            normal & \xmark & 2.30 & $3.27$ & $0.28$ & $3.55$ & $1.88$ & $4.35$ & $0.49$\\
            \hline
            laplace & \xmark & 3.00 & $2.